## Segmenting and Clustering Neighborhoods in Toronto - 2nd Part

This is the notebook that contains the code for the seconde part of the week 3 assignment of the course Applied Data Science Capstone. 

In order to parse the HTML page, I decided to use __*BeautifulSoup*__

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
# Read the URL page at Wikipedia
data = BeautifulSoup(page.text, "html.parser")

# Get the HTML table inside the HTML data read
table = data.find('table')

# Get all the rows of the HTML table
table_rows = table.find_all('tr')

# Get the header TH
header = table_rows[0].find_all('th')

# Get the column values (TD) of the header TH, which will be the column labels of the dataframe
labels = [table_rows[0].text for table_rows[0] in header]
# Clean the '\n' special character
labels = np.char.replace(labels, '\n', '')

# Create the dataframe
# The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns = labels)

# Get all the other columns (TD) from the rows (TR)
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    # Ignore the first row (header) and the rows which have "Not assigned" boroughs
    # Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
    if (row != [] and row[1] != "Not assigned\n"):
        row = pd.Series(row,labels)
        df = df.append([row],ignore_index=True)     

# Clean the data removing '\n'        
df.replace('\n','', regex=True, inplace=True)   

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
arr = np.where(pd.isnull(df['Neighborhood']))
print(arr)
# there are no 'not assigned' neighborhoods in the dataframe



(array([], dtype=int64),)


In [6]:
# Read the geospatial data csv
geospatial = pd.read_csv("http://cocl.us/Geospatial_data") 
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
# Change the dataframes indexes, in order to make it possible to merge them later
df.set_index('Postal Code', inplace=True)
geospatial.set_index('Postal Code', inplace=True)

In [8]:
# Merge both dataframes
mergedDf = df.merge(geospatial, left_index=True, right_index=True)
mergedDf

,Borough,Neighborhood,Latitude,Longitude
Postal Code,,,,
M3A,North York,Parkwoods,43.753259,-79.329656
M4A,North York,Victoria Village,43.725882,-79.315572
M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M3B,North York,Don Mills,43.745906,-79.352188
M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937


In [9]:
# Transform Postal Code into a column again
mergedDf['CEP'] = mergedDf.index
mergedDf.reset_index(inplace=True)
del mergedDf['Postal Code']
mergedDf.rename(columns={'CEP':'Postal Code'}, inplace=True)

mergedDf.head()

,Borough,Neighborhood,Latitude,Longitude,Postal Code
0,North York,Parkwoods,43.753259,-79.329656,M3A
1,North York,Victoria Village,43.725882,-79.315572,M4A
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,M5A
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,M6A
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,M7A


In [10]:
# Define Postal Code as the first column
cols = mergedDf.columns.tolist()
cols = cols[-1:] + cols[:-1]
mergedDf = mergedDf[cols]
mergedDf.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [11]:
mergedDf


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
